### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-07-31


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-07-31
Yesterday: 2025-07-30


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-07-30'
ORDER BY a.name



(11, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-07-30' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,DIF,2025-07-23,2025-07-30,8.15,8.20,60009200,8.20,8.10,B,SET,0.05,0.61
1,GVREIT,2025-07-25,2025-07-30,6.55,6.50,728440,6.55,6.50,B,SET,-0.05,-0.76
2,JMT,2025-07-29,2025-07-30,10.50,11.10,55174650,11.10,10.40,B,SET50,0.60,5.71
3,SENA,2025-07-24,2025-07-30,1.66,1.78,1131740,1.78,1.65,B,SET,0.12,7.23
4,WHAIR,2025-07-30,2025-07-30,5.50,5.45,12048560,5.50,4.96,B,SET,-0.05,-0.91
5,WHART,2025-07-25,2025-07-30,9.35,9.30,952616,9.45,9.30,B,SET,-0.05,-0.53
6,AIMIRT,2025-07-30,2025-07-30,9.55,9.65,407346,9.70,9.55,S,SET,0.10,1.05
7,IVL,2025-07-25,2025-07-30,21.30,23.10,26891084,23.10,21.10,S,SET50,1.80,8.45
8,MCS,2025-07-30,2025-07-30,8.30,8.35,974502,8.40,8.20,S,SET,0.05,0.60
9,NER,2025-07-25,2025-07-30,4.24,4.40,9808156,4.40,4.22,S,SET,0.16,3.77


In [20]:
df.shape

(11, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,DIF,2025-07-23,2025-07-30,8.15,8.20,"60,009,200",8.20,8.10,0.61%,B
1,GVREIT,2025-07-25,2025-07-30,6.55,6.50,"728,440",6.55,6.50,-0.76%,B
2,JMT,2025-07-29,2025-07-30,10.50,11.10,"55,174,650",11.10,10.40,5.71%,B
3,SENA,2025-07-24,2025-07-30,1.66,1.78,"1,131,740",1.78,1.65,7.23%,B
4,WHAIR,2025-07-30,2025-07-30,5.50,5.45,"12,048,560",5.50,4.96,-0.91%,B
5,WHART,2025-07-25,2025-07-30,9.35,9.30,"952,616",9.45,9.30,-0.53%,B
6,AIMIRT,2025-07-30,2025-07-30,9.55,9.65,"407,346",9.70,9.55,1.05%,S
7,IVL,2025-07-25,2025-07-30,21.30,23.10,"26,891,084",23.10,21.10,8.45%,S
8,MCS,2025-07-30,2025-07-30,8.30,8.35,"974,502",8.40,8.20,0.60%,S
9,NER,2025-07-25,2025-07-30,4.24,4.40,"9,808,156",4.40,4.22,3.77%,S


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(11, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(11, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,DIF,2025-07-23,2025-07-30,8.15,8.20,60009200,8.20,8.10,0.61,B,8.15,-0.05,8082306,2025-07-31
1,GVREIT,2025-07-25,2025-07-30,6.55,6.50,728440,6.55,6.50,-0.76,B,6.55,0.05,224900,2025-07-31
2,JMT,2025-07-29,2025-07-30,10.50,11.10,55174650,11.10,10.40,5.71,B,11.10,0.00,28129130,2025-07-31
3,SENA,2025-07-24,2025-07-30,1.66,1.78,1131740,1.78,1.65,7.23,B,1.80,0.02,272948,2025-07-31
4,WHAIR,2025-07-30,2025-07-30,5.50,5.45,12048560,5.50,4.96,-0.91,B,5.40,-0.05,87459,2025-07-31
5,WHART,2025-07-25,2025-07-30,9.35,9.30,952616,9.45,9.30,-0.53,B,9.10,-0.20,2043353,2025-07-31
6,AIMIRT,2025-07-30,2025-07-30,9.55,9.65,407346,9.70,9.55,1.05,S,9.85,0.20,631525,2025-07-31
7,IVL,2025-07-25,2025-07-30,21.30,23.10,26891084,23.10,21.10,8.45,S,23.10,0.00,18279560,2025-07-31
8,MCS,2025-07-30,2025-07-30,8.30,8.35,974502,8.40,8.20,0.60,S,8.40,0.05,306635,2025-07-31
9,NER,2025-07-25,2025-07-30,4.24,4.40,9808156,4.40,4.22,3.77,S,4.42,0.02,6997991,2025-07-31


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-07-30 2025-07-31   9.55   9.85   3.14%     407,346   9.85   9.55 S Update
DIF      2025-07-31 2025-07-31   8.20   8.15  -0.61%  60,009,200   8.20   8.10 B Insert
GVREIT   2025-07-31 2025-07-31   6.50   6.55   0.77%     728,440   6.55   6.50 B Insert
IVL      2025-07-25 2025-07-31  21.30  23.10   8.45%  26,891,084  23.10  21.10 S Update
JMT      2025-07-29 2025-07-31  10.50  11.10   5.71%  55,174,650  11.10  10.40 B Update
MCS      2025-07-30 2025-07-31   8.30   8.40    1.2%     974,502   8.40   8.20 S Update
NER      2025-07-25 2025-07-31   4.24   4.42   4.25%   9,808,156   4.42   4.22 S Update
PTG      2025-07-31 2025-07-31   6.85   6.75  -1.46%   7,745,050   6.85   6.40 S Insert
SENA     2025-07-24 2025-07-31   1.66   1.80   8.43%   1,131,740   1.80   1.65 B Update
WHAIR    2025-07-30 2025-07-31  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-07-31' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(11, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,DIF,2025-07-31,2025-07-31,8.20,8.15,"60,009,200",8.20,8.10,B,SET,-0.05,-0.61%
1,GVREIT,2025-07-31,2025-07-31,6.50,6.55,"728,440",6.55,6.50,B,SET,0.05,0.77%
2,JMT,2025-07-29,2025-07-31,10.50,11.10,"110,349,300",11.10,10.40,B,SET50,0.60,5.71%
3,SENA,2025-07-24,2025-07-31,1.66,1.80,"2,263,480",1.80,1.65,B,SET,0.14,8.43%
4,WHAIR,2025-07-30,2025-07-31,5.50,5.40,"24,097,120",5.50,4.96,B,SET,-0.10,-1.82%
5,WHART,2025-07-25,2025-07-31,9.35,9.10,"1,905,232",9.45,9.10,B,SET,-0.25,-2.67%
6,AIMIRT,2025-07-30,2025-07-31,9.55,9.85,"814,692",9.85,9.55,S,SET,0.30,3.14%
7,IVL,2025-07-25,2025-07-31,21.30,23.10,"53,782,168",23.10,21.10,S,SET50,1.80,8.45%
8,MCS,2025-07-30,2025-07-31,8.30,8.40,"1,949,004",8.40,8.20,S,SET,0.10,1.20%
9,NER,2025-07-25,2025-07-31,4.24,4.42,"19,616,312",4.42,4.22,S,SET,0.18,4.25%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
2,JMT,2025-07-29,2025-07-31,10.50,11.10,"110,349,300",11.10,10.40,B,SET50,0.60,5.71%
3,SENA,2025-07-24,2025-07-31,1.66,1.80,"2,263,480",1.80,1.65,B,SET,0.14,8.43%
7,IVL,2025-07-25,2025-07-31,21.30,23.10,"53,782,168",23.10,21.10,S,SET50,1.80,8.45%


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-07-31 20:46:40
